In [1]:
import os

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [3]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "llama-3.1-8b-instant")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000027D8DF916A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000027D8DF923C0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_community.chat_message_histories import ChatMessageHistory             # stores conversation per session
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory               # Connects model + history → adds memory

In [5]:
# Creating function to keep seperate sessions 

store = {}
def get_session_history(session_id: str)-> BaseChatMessageHistory:
   if session_id not in store:
      store[session_id] = ChatMessageHistory()
   return store[session_id]

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([("system", "You are an helpful assistant. Answer all the questions to the best of your ability in {language}"),
MessagesPlaceholder(variable_name = "messages",)])

In [8]:
chain = prompt | model

In [14]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key = "messages")

In [15]:
config = {'configurable': {'session_id': 'chat1'}}

In [16]:
from langchain_core.messages import HumanMessage

In [17]:
response = with_message_history.invoke({"messages": [HumanMessage(content = "Hi, My name is Nidhi and I am a GenAI Engineer")], 
"language": "Hindi"}, config = config)

response.content

'नमस्ते निधि जी, मैं आपकी सहायता के लिए तैयार हूँ। एक जेनरेटिव एल्गोरिथम इंजीनियर के रूप में, मुझे लगता है कि आप काफी रोचक काम कर रही होंगी। आप किस क्षेत्र में काम कर रही हैं?'

In [18]:
response = with_message_history.invoke({"messages": [HumanMessage(content = "What's my name")], 
"language": "Hindi"}, config = config)

response.content

'आपका नाम निधि है।'

In [20]:
config1 = {'configurable': {'session_id': 'chat1'}}

In [21]:
response = with_message_history.invoke({"messages": [HumanMessage(content = "What's my name")], 
"language": "Hindi"}, config = config1)

response.content

'आपका नाम निधि है।'

In [22]:
config1 = {'configurable': {'session_id': 'chat2'}}

In [23]:
response = with_message_history.invoke({"messages": [HumanMessage(content = "What's my name")], 
"language": "Hindi"}, config = config1)

response.content

'मुझे खेद है, लेकिन मैं आपके व्यक्तिगत नाम नहीं जानता। हमारे पिछले संवाद में भी यह जानकारी नहीं थी। कृपया अपना नाम बताएं ताकि हम आगे की बातचीत कर सकें।'